###### 2_dbscan_clustering.ipynb

## 디비스캔 (DBSCAN)
- 밀도 기반 군집 알고리즘
- 입실론 반경과 최소 포인트를 매개변수로 사용하여 군집을 정의
- 노이즈(이상치)를 자동으로 분류

In [ ]:
import numpy as np

# 샘플데이터
x = np.array([
  [1,2], [1,3], [2,2], [2,3],     # Cluster 1
  [8,8], [8,9], [9,8], [9,9],     # Cluster 2
  [5,5]                           # Noise (고립점)
])

# 모델
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=1.5, min_samples=3)
# * eps : 반경 (이웃을 찾을 거리)
#         한 점을 중심으로 해당 거리 안에 있는 점들을 이웃으로 간주
#   - 기본 가이드라인 (참고)
#     + 2D 저차원 : 0.3 ~ 1.0
#     + 5D ~ 10D  : 1.0 ~ 5.0
#     + 10D 이상 고차원 : 5.0 이상
#  => 데이터 분포와 스케일에 따라 달라짐!

# * min_samples : 핵심점 판단을 위한 최소 포인트 수
#                 한 점이 핵심점이 되려면 주변 eps 반경 안에 min_samples 이상의 점이 있어야 함
#   - 기본 가이드라인 (참고)
#     + 저차원 : 3 ~ 5
#     + 고차원 : 차원수 + 1